## Przygotowanie sesji Spark
Zainicjowanie sesji Spark oraz stworzenie schematu bazy danych z której będziemy korzystać.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

Pobranie sesji Spark jest proste dla programisty korzystającego z notatnika, wymaga podania tylko kilku parametrów, ale faktyczna konfiguracja jest bardziej rozbudowana

In [ ]:
import os
args = os.environ['PYSPARK_SUBMIT_ARGS'].replace("  ", "\n")
print(args)

Mamy pobraną sesję sparkową. Powstały dodatkowe pody gotowe na realizację obliczeń. A jak zwolnić te zasoby?

In [ ]:
spark.stop()

Ponownie pobieramy sesję Spark. Będziemy z niej korzystać. Po zakończonej pracy należy pamiętać o zastopowaniu sesji.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

## Odczyt danych
Korzystając z sesji Spark można odczytać dane zapisane w lokalizacji dostępnej dla executorów (procesów obliczeniowych) koordynowanych przez Spark. 
Konieczne jest podanie ścieżki dostępowej do pliku i formatu danych (nie jest to jednoznazne z rozszerzeniem pliku).
Dystrybucja Spark udostępnia kilka tzw Data Sources, które odczytują i zapisują dane w określonych formatach (CSV, formaty kolumnowe: parquet/orc).
Data Sources zgodne z opracowanym interfejsem można samodzielnie tworzyć. Na dzisiejszych zajęciach będziemy korzystać z takich zdefiniowanych DS:
* FASTQDataSource
* BAMDataSource
* VCFDataSource

**UWAGA - powyższe sposoby odczytu plików nie są częścią głównej dystrybucji Spark. Wymagana jest dodatkowa konfiguracja.**

In [ ]:
import os                               # moduł OS języka Python
user_name = os.environ.get('USER')      # pobieramy zmienną środowiskową USER
bucket = f"gs://edugen-lab-{user_name}" # konstruujemy sciezke dostepowa do pliku
print(bucket)

---

`Co oznacza f przed cudzysłowem?`

`Czy jest różnica między stosowaniem apostrofu i cudzysłowu przy definicji zmiennych przechowywujących łańcuchy znaków?`

In [ ]:
reads_path = f"{bucket}/fastq/*"  # * oznacza wszystkie pliki we wskazanej lokalizacji. Można podać konkretny plik
fastq_all = spark.read.load(reads_path, format="org.biodatageeks.sequila.datasources.FASTQ.FASTQDataSource")

### Weryfikacja danych

In [ ]:
type(fastq_all)      # jaki jest typ danych utworzonej zmiennej? 

In [ ]:
fastq_all.printSchema() # jaki jest schemat danych?

In [ ]:
len(fastq_all.columns)           # wymiary (liczba kolumn)

In [ ]:
fastq_all.count()               # wymiary (liczba wierszy)

In [ ]:
fastq_all.explain(True)              #  plan wykonania

DataFrame jest abstrakcją nad innym typem danych (RDD), który jest podstawową rozproszoną strukturą danych. Poprzez DF możemy dostać się do rdd i zweryfikować na przykład liczbę partycji danych.

In [ ]:
fastq_all.rdd.getNumPartitions() # liczba partycji (bloków danych)

### Podgląd danych

In [ ]:
fastq_all.show()

In [ ]:
fastq_all.show(5)  # pierwsze 5 wierszy

In [ ]:
fastq_all.show(truncate=False) # bez skracania zawartości kolumn

Widok "szerokich" tabel jest nieczytelny, w kolejnych częściach warsztatów zaradzimy temu korzystając z dodatkowej biblioteki.

## Dostęp do wybranych danych

#### Operacja projekcji (SELECT)

In [ ]:
fastq_all.select("sample_id").show()

`Czy operacja select (sample_id) wpłynęła na oryginalny data frame fastq_all?`

In [ ]:
fastq_all.printSchema()
fastq_all.show() 

Jeśli chcemy zachować wynik działania transformacji (w celu późniejszego wykorzystania) trzeba wynik zachować w zmiennej.

In [ ]:
fastq_sample_only = fastq_all.select("sample_id")  

In [ ]:
fastq_sample_only.printSchema()
fastq_sample_only.show()

In [ ]:
fastq_all.select("sample_id","seq").show()

Powiedzmy, że interesują nas wszystkie kolumny poza qual. Jak to zrobic? Można wylistować wszystkie kolumny poza qual, ale to uciazliwe. Mozna skorzystac z operacji drop.

In [ ]:
fastq_no_qual=fastq_all.drop('qual')

`Czy operacja usunięcia kolumny qual wpłynęła na oryginalny data frame fastq_all?`

In [ ]:
fastq_no_qual.printSchema()
fastq_no_qual.show()

## <span style='background:yellow'> ZADANIE  </span>

Stwórz nową ramkę danych która będzie zawierać wszystkie kolumny z fastq_all poza pos_x i pos_y. Wyświetl 10 wierszy z tej kolekcji pokazując pełną zawartość kolumn

In [ ]:
fastq_cropped = fastq_all.drop('pos_x', 'pos_y')
fastq_cropped.show(10, truncate=False)

## Wartości unikalne
Jeśli chcemy uzyskać unikalne wartości w określonych kolumnach korzystamy z metody distinct().
Operacje na DF można łańcuchowo łączyć, zatem na wyniku działania select() można wywołać kolejne metody.

In [ ]:
fastq_all.select('sample_id').distinct().show()

In [ ]:
fastq_all.select('sample_id', 'filter_passed').distinct().show()

In [ ]:
fastq_all.count()

# Sortowanie

Do sortowania służy metoda orderBy. Domyślne sortowanie jest rosnące.

In [ ]:
fastq_all.select('sample_id').distinct().orderBy('sample_id').show()

In [ ]:
fastq_all.select('sample_id').distinct().orderBy('sample_id', ascending=False).show()

In [ ]:
fastq_all.orderBy('sample_id', 'seq', ascending=False).show()  # kierunek sortowania jest wspólny dla listy kolumn

In [ ]:
fastq_all.orderBy('sample_id', ascending=False).orderBy('seq', ascending=True).show() # sortowanie malejace i rosnące na dwóch roznych kolumnach 

### Filtrowanie wynikow
Nasz zbiór danych posiada odczyty z 3 próbek. Ograniczmy się do wybranych próbek.

In [ ]:
fastq_mother = fastq_all.filter("sample_id = 'mother'")

`Czy już odbył się odczyt danych z fastq?`

In [ ]:
fastq_mother.show()

In [ ]:
fastq_mother.select('sample_id').distinct().show()

In [ ]:
fastq_mother.count()

Warunki można łączyć spójnikami logicznymi. 
Można używać
* operatorów arytmetycznych (=, !=, >, >=, <, <=)
* przynależności do zbioru (IN/NOT IN) 
* porównania znaków (LIKE/NOT LIKE)
* przyrównania do wartości NULL (IS NULL/ IS NOT NULL)

Przy korzystaniu z LIKE można użyć % jako oznaczenie dowolnego ciągu znaków.

Konstrukcja warunku w metodzie filter() jak taka jak w klauzuli WHERE W SQL.

Pokaż odczyty spełniające warunek ze nazwa instrumentu jest pusta, run_id jest >=0 a odczyt zaczyna sie od liter GCA. Pokaz tylko kolumny z filtra oraz nazwe probki

In [ ]:
fastq_mother.filter('instrument_name IS NULL and run_id>=0 and seq LIKE "GCA%"').select('sample_id', 'instrument_name', 'run_id', 'seq').show()

In [ ]:
fastq_mother.select('sample_id', 'instrument_name', 'run_id', 'seq').filter('instrument_name IS NULL and run_id>=0 and seq LIKE "GCA%"').show() # kolejnosc select i filter bez znaczenia

## <span style='background:yellow'> ZADANIE  </span>

Napisz polecenie które policzy ile jest rekordów dla próbki syna które spełniają warunki, że sekwencja odczytu konczy się na TGG a qual zaczyna się od ==.

In [ ]:
fastq_all.filter("sample_id='son' and seq LIKE '%TGG' and qual LIKE '==%' ").count()

## Uzywanie funkcji, kolumny wyliczane

Dostępne są funkcje skalarne (przykład: ROUND, UPPER, CURRENT_DATE) oraz agregujące (MIN, MAX, AVG, SUM, COUNT).
Niektóre funkcje są dostępne "od razu" bez dodatkowych poleceń import. 
Lista funkcji znajduje się : https://spark.apache.org/docs/latest/api/sql/index.html 

In [ ]:
fastq_all.selectExpr("*").show()  # pokaż wszystkie kolumny tego DF

Dodanie dwóch dodatkowych kolumn wyliczanych 

In [ ]:
fastq_all.selectExpr("*", "length(seq) as len_seq", "length(qual) as len_qual" ).show() ## dodanie dwóch kolumn wyliczanych przy uzyciu funkcji LENGTH

Alias - nadanie kolumnie lub kolumnie wyliczanej nazwy

In [ ]:
extended_fastq = fastq_all.selectExpr("*", "length(seq) as len_s", "length(qual) as len_q" ).drop('instrument_name', 'flowcell_id', 'filter_passed','control_num') # AS alias

In [ ]:
extended_fastq.printSchema()

Dodanie nowej kolumny, dla każdego wiersza zostanie dodana wartość zwracana przez funkcję current_date()

In [ ]:
from pyspark.sql.functions import current_date
extended_fastq.withColumn ("date", current_date()).show()

Dodanie kolumny o stałej wartości dla każdej wartości wymaga wykorzystania funkcji lit (), która przekształci stała wartość w kolumnę.

In [ ]:
from pyspark.sql.functions import lit

extended_fastq.withColumn("imported_by", lit(user_name)).withColumn("format", lit('FASTQ')).show()

## <span style='background:yellow'> ZADANIE  </span>

Napisz polecenie które stworzy ramkę danych zawierającą sklejenie wartości dwóch kolumn (sample_id) oraz daty eksperymentu (dodaj kolumne z wartościami 2019-01-15) . W wynikach chcemy mieć tylko dane matki i ojca. Kolumny wynikowe: nazwa próbki, seq, qual, data eksperymentu oraz scalona nazwa probki oraz data eksperymenty (np father-2019-01-15). Posortuj po nazwie próbki. Pokaż schemat ramki. Upewnij się, że data eksperymentu jest typu date. 

* Zwróć uwagę na potrzebę konwersji ciągu znaków na datę

In [ ]:
from pyspark.sql.functions import to_date, lit, concat

df = fastq_all.withColumn("experiment_date", to_date(lit('2019-01-15')))\
        .withColumn("sample_date", concat("sample_id", concat(lit("-"), "experiment_date")))\
        .select ("sample_id", "sample_date", "seq", "qual", "experiment_date")\
        .filter ("sample_id IN ('mother', 'father')") \
        .orderBy("sample_id")
df.printSchema()
df.show()

### Instrukcje warunkowe przy kolumnach wyliczanych

In [ ]:

fastq_dates=fastq_all.selectExpr('*', 'if(sample_id = "son",to_date("2018-11-10"), to_date("2019-01-15")) as experiment_date')

## <span style='background:yellow'> ZADANIE  </span>

Napisz polecenia, które zweryfikuje czy daty eksperymentów zostały dodane poprawnie

In [ ]:
fastq_dates.printSchema()
fastq_dates.show()
fastq_dates.select("sample_id", "experiment_date").show()
fastq_dates.select("sample_id", "experiment_date").distinct().show()

## Grupowanie

In [ ]:
fastq_all.groupBy("sample_id").show()

In [ ]:
type(fastq_all.groupBy("sample_id")) # to nie jest DF

In [ ]:
sample_count=fastq_all.groupBy("sample_id").count()

In [ ]:
type(sample_count)

In [ ]:
sample_count.show()

In [ ]:
sample_count.orderBy("count").show()

## <span style='background:yellow'> ZADANIE  </span>

Napisz funkcje, ktora znajdzie rozklady jakosci dla 1, 2, 3 i 4 pozycji odczytu.
*Nastepnie zaprezentuj wyniki w postaci serii histogramów

In [ ]:
quals = fastq_all.selectExpr( "ascii(SUBSTR(qual,1)) -33 as p1", 
                                    "ascii(SUBSTR(qual,2))-33 as p2",
                                    "ascii(SUBSTR(qual,3))-33 as p3",
                                    "ascii(SUBSTR(qual,4))-33 as p4")


import matplotlib.pyplot as plt
quals.toPandas().boxplot()

### Mapowanie do genomu referencyjnego
Wykonamy mapowanie do genomu referencyjnych korzystając z rozproszenia danych miedzy procesy obliczeniowe sparka.

Przygotowanie ścieżek do plików.

In [ ]:
import os
user_name = os.environ.get('USER')
bucket = f"gs://edugen-lab-{user_name}"

reads_file_path = f"{bucket}/fastq/mother.fastq"
ref_path = "/mnt/data/mapping/ref/ref.fasta"

Konstruujemy komendę, która będzie uruchamiana na procesach obliczeniowych. Potrzebne narzędzia muszą być dostępne na węzłach obliczeniowych.

In [ ]:
command = f'bwa mem -p {ref_path} - | samtools fixmate -m - - | samtools sort  | samtools markdup -r -S - -  | samtools addreplacerg  -r "ID:S1" -r "SM:S1"  -r "PL:ILLUMINA" - | samtools view -b -'

Żeby wykonać rozproszone obliczenia na danych genomicznych nalezy wykorzystac dodatkową bibliotekę.

In [ ]:
from pyseqtender import SeqTenderAlignment

seq_aligner = SeqTenderAlignment(spark, reads_file_path, command)
alignments_rdd = seq_aligner.pipe_reads()

Zapisujemy plik na kubełek.

In [ ]:
bam_file_path = f"{bucket}/bam/mother10.bam"
seq_aligner.save_reads(bam_file_path, alignments_rdd)

In [ ]:
!gsutil ls gs://edugen-lab-$USER/bam

## <span style='background:yellow'> ZADANIE  </span>

Na podstawie notatników z zajęć z genomiki wyświetl fragment pliku BAM w widge'cie IGV. 

In [ ]:
# najpierw trzeba upublicznić kubełek
!gsutil iam ch allUsers:objectViewer $bucket

from igv_jupyterlab import IGV
igv = IGV(genome="hg19" )
display(igv)
import time
time.sleep(3)
t1 = igv.create_track(
        name = "mother BAM",
        url = f"{bucket}/bam/mother10.bam",
        index_url= f"{bucket}/bam/mother10.bam.bai",
        indexed= True
)

igv.load_track(t1)

igv.search_locus("20", 10002294, 10002623)

Kończymy notatniki, należy zamknąć sesję.

In [ ]:
spark.stop()